In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from math import pi

In [4]:
df = pd.read_csv('data/train.csv').dropna()
df.drop(columns=['center_id', 'patient_id', 'image_num'], inplace=True)
df.head()

,image_id,label
0,006388_0,CE
1,008e5c_0,CE
2,00c058_0,LAA
3,01adc5_0,LAA
4,026c97_0,CE


In [5]:
CE_count = df.where(df['label'] == 'CE').count()[0]
CE_count

547

In [6]:
LAA_count = df.where(df['label'] == 'LAA').count()[0]
LAA_count

207

In [7]:
cutoff = CE_count if CE_count < LAA_count else LAA_count
cutoff

207

In [8]:
CE = df.where(df['label'] == 'CE').dropna().head(cutoff)
LAA = df.where(df['label'] == 'LAA').dropna().head(cutoff)

In [9]:
data = pd.concat([CE, LAA]).dropna()
data.head(cutoff*2)

,image_id,label
0,006388_0,CE
1,008e5c_0,CE
4,026c97_0,CE
6,029c68_0,CE
7,032f10_0,CE
...,...,...
740,fd684f_0,LAA
745,fde9ba_0,LAA
750,fe9bec_0,LAA
752,ffec5c_0,LAA


In [10]:
images = np.array([np.array(Image.open(f'jpgs/{img_id}.jpg')) for img_id in data['image_id']])
images.shape

(414, 512, 512)

In [11]:
enc_labels = np.zeros(shape=(414, 2))
for i, label in enumerate(data['label']):
    enc_labels[i][1 if label == 'CE' else 0] = 1

In [12]:
keras.utils.set_random_seed(1234)
ConvNN = keras.Sequential()
ConvNN.bias = 0.5
ConvNN.add(keras.layers.Input((512, 512)))
ConvNN.add(keras.layers.Rescaling(1.0/255))

ConvNN.add(keras.layers.RandomFlip(
    mode="horizontal_and_vertical", seed=1234
))

ConvNN.add(keras.layers.RandomRotation(
    (-0.5, 0.5),
    fill_mode="constant",
    interpolation="bilinear",
    seed=1234
))

ConvNN.add(keras.layers.RandomZoom(
    (-0.2, 0.01),
    width_factor=(-0.2, 0.01),
    fill_mode="constant",
    interpolation="bilinear",
    seed=1234
))

ConvNN.add(keras.layers.Convolution1D(256, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Flatten())
ConvNN.add(keras.layers.Dense(262144, activation='relu'))
ConvNN.add(keras.layers.Dense(512, activation='relu'))
ConvNN.add(keras.layers.Dense(256,  activation='relu'))

ConvNN.add(keras.layers.Dropout(.5))

ConvNN.add(keras.layers.Dense(2, activation='softmax'))
ConvNN.output_shape

(None, 2)

In [13]:
callbacksConv = [
    keras.callbacks.ModelCheckpoint("convNNv2_save_at_{epoch}.h5"),
]

In [14]:
ConvNN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [15]:
ConvNN.fit(images, enc_labels, epochs=25, callbacks = callbacksConv)

Epoch 1/25
13/13 [==============================] - 110s 8s/step - loss: 1.2711 - accuracy: 0.5217
Epoch 2/25
13/13 [==============================] - 84s 7s/step - loss: 0.7000 - accuracy: 0.4662
Epoch 3/25
13/13 [==============================] - 94s 8s/step - loss: 0.7054 - accuracy: 0.4879
Epoch 4/25
13/13 [==============================] - 104s 8s/step - loss: 0.6935 - accuracy: 0.5338
Epoch 5/25
13/13 [==============================] - 94s 8s/step - loss: 0.7218 - accuracy: 0.4928
Epoch 6/25
13/13 [==============================] - 99s 8s/step - loss: 0.7037 - accuracy: 0.4710
Epoch 7/25
13/13 [==============================] - 82s 7s/step - loss: 0.6936 - accuracy: 0.4976
Epoch 8/25
13/13 [==============================] - 82s 7s/step - loss: 0.7011 - accuracy: 0.4734
Epoch 9/25
13/13 [==============================] - 94s 8s/step - loss: 0.6942 - accuracy: 0.5024
Epoch 10/25
13/13 [==============================] - 96s 8s/step - loss: 0.6938 - accuracy: 0.5048
Epoch 11/25
13/13

KeyboardInterrupt: 